In [2]:
import sys
sys.path.append("../src")

In [3]:
from CurvyCUSIPs.CurveDataFetcher import CurveDataFetcher

from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy

from functools import reduce
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
import statsmodels.api as sm

import urllib.parse
import requests
import os
from dotenv import dotenv_values
env_path = os.path.join(os.getcwd(), "../.env")
print(env_path)
config = dotenv_values(env_path)

import nest_asyncio
nest_asyncio.apply()

import matplotlib.pylab as pylab
params = {
    "legend.fontsize": "x-large",
    "axes.labelsize": "x-large",
    "axes.titlesize": "x-large",
    "xtick.labelsize": "x-large",
    "ytick.labelsize": "x-large",
}
pylab.rcParams.update(params)
sns.set(style="whitegrid", palette="dark")
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

%load_ext autoreload
%autoreload 2

c:\Users\chris\Project Bond King\Curvy-CUSIPs\notebooks\../.env


In [4]:
curve_data_fetcher = CurveDataFetcher(use_ust_issue_date=True, fred_api_key=config["FRED_API_KEY"])

In [5]:
as_of_date = datetime(2024, 10, 31)
sdr = curve_data_fetcher.dtcc_sdr_fetcher.fetch_dtcc_sdr_data_timeseries(
    start_date=as_of_date , end_date=as_of_date, agency="CFTC", asset_class="RATES"
)
sdr_df = sdr[as_of_date]

FETCHING DTCC SDR DATASETS...: 100%|██████████| 1/1 [00:00<00:00,  1.10it/s]


In [7]:
swaption_dsb_lookup_df = pd.read_csv(r"C:\Users\chris\Project Bond King\Curvy-CUSIPs\src\CurvyCUSIPs\dsb_upis\dsb_swaption_upis.csv")
capfloor_dsb_lookup_df = pd.read_csv(r"C:\Users\chris\Project Bond King\Curvy-CUSIPs\src\CurvyCUSIPs\dsb_upis\dsb_capfloor_upis.csv")
fixed_float_swaps_dsb_lookup_df = pd.read_csv(r"C:\Users\chris\Project Bond King\Curvy-CUSIPs\src\CurvyCUSIPs\dsb_upis\dsb_fixed_float_swaps_upis.csv")

In [8]:
capfloor_cols = [
    "Event timestamp",
    "Execution Timestamp",
    "Effective Date",
    "Expiration Date",
    "Notional amount-Leg 1",
    "First exercise date",
    "Option Premium Amount",
    "Strike Price",
    "Unique Product Identifier",
    "UPI FISN",
    "UPI Underlier Name",
]


def format_capfloor_time_and_sales(df: pd.DataFrame):
    date_columns = [
        "Event timestamp",
        "Execution Timestamp",
        "Effective Date",
        "Expiration Date",
        "First exercise date",
    ]
    for col in date_columns:
        df[col] = pd.to_datetime(df[col])
        df[col] = df[col].dt.tz_localize(None)

    df = df[df["Event timestamp"].dt.date >= as_of_date.date()]
    df = df[df["Execution Timestamp"].dt.date >= as_of_date.date()]

    year_day_count = 365
    df["Forward"] = ((df["Effective Date"] - df["Execution Timestamp"]).dt.days / year_day_count).round()
    df["Expiry (d)"] = (df["Expiration Date"] - df["Effective Date"]).dt.days
    df["Expiry (m)"] = df["Expiry (d)"] / 30
    df["Expiry (yr)"] = df["Expiry (d)"] / year_day_count

    df["Notional amount-Leg 1"] = pd.to_numeric(df["Notional amount-Leg 1"].str.replace(",", ""), errors="coerce")
    df["Option Premium Amount"] = pd.to_numeric(df["Option Premium Amount"].str.replace(",", ""), errors="coerce")
    df["Option Premium Amount / $mm Notational"] = df["Option Premium Amount"] / (
        df["Notional amount-Leg 1"] / 1_000_000
    )
    df["Strike Price"] = pd.to_numeric(df["Strike Price"], errors="coerce")
    df = df.dropna(
        subset=["Notional amount-Leg 1", "Option Premium Amount", "Strike Price"]
    )
    df = df[df["Option Premium Amount"] != 0]
    
    return df

In [9]:
cap_conds = (
    # USD SOFR OIS COMPOUNDED 1D
    (capfloor_dsb_lookup_df["Derived_CFIOptionStyleandType"] == "European-Call") &
    # (capfloor_dsb_lookup_df["Attributes_UnderlyingInstrumentIndex"] == "USD-SOFR-OIS Compound") &
    # (capfloor_dsb_lookup_df["Attributes_UnderlyingInstrumentIndex"] == "USD-SOFR-COMPOUND") &
    (capfloor_dsb_lookup_df["Attributes_UnderlyingInstrumentIndex"] == "USD-SOFR CME Term") &
    # (capfloor_dsb_lookup_df["Attributes_UnderlyingInstrumentIndexTermValue"] == 1) &
    # (capfloor_dsb_lookup_df["Attributes_UnderlyingInstrumentIndexTermUnit"] == "DAYS") &
    (capfloor_dsb_lookup_df["Attributes_NotionalCurrency"] == "USD") &
    ((capfloor_dsb_lookup_df["Attributes_DeliveryType"] == "CASH") | (capfloor_dsb_lookup_df["Attributes_DeliveryType"] == "PHYS"))
)

cap_rows = capfloor_dsb_lookup_df[cap_conds]
cap_rows 

,TemplateVersion,Header_AssetClass,Header_InstrumentType,Header_UseCase,Header_Level,Identifier_UPI,Identifier_Status,Identifier_StatusReason,Identifier_LastUpdateDateTime,Derived_ClassificationType,...,Derived_OptionExerciseStyle,Derived_ValuationMethodorTrigger,Derived_CFIOptionStyleandType,Derived_CFIDeliveryType,Attributes_NotionalCurrency,Attributes_UnderlyingInstrumentIndex,Attributes_UnderlyingInstrumentIndexTermValue,Attributes_UnderlyingInstrumentIndexTermUnit,Attributes_OptionType,Attributes_DeliveryType
96,1,Rates,Option,CapFloor,UPI,QZ1HJ2WJ730F,New,NaN,2024-09-12T23:27:33,HRMAMP,...,EURO,Other,European-Call,Physical,USD,USD-SOFR CME Term,4,MNTH,CALL,PHYS
136,1,Rates,Option,CapFloor,UPI,QZSG0HNZK9M6,New,NaN,2024-11-01T18:22:40,HRMAMC,...,EURO,Other,European-Call,Cash,USD,USD-SOFR CME Term,2,YEAR,CALL,CASH
581,1,Rates,Option,CapFloor,UPI,QZZ0KX4QM65W,New,NaN,2024-01-28T17:48:09,HRMAMC,...,EURO,Other,European-Call,Cash,USD,USD-SOFR CME Term,90,DAYS,CALL,CASH
584,1,Rates,Option,CapFloor,UPI,QZ6WKJ508599,New,NaN,2024-01-28T17:28:18,HRMAMC,...,EURO,Other,European-Call,Cash,USD,USD-SOFR CME Term,30,DAYS,CALL,CASH
685,1,Rates,Option,CapFloor,UPI,QZDPMT72JBX8,New,NaN,2024-01-28T19:47:13,HRMAMC,...,EURO,Other,European-Call,Cash,USD,USD-SOFR CME Term,180,DAYS,CALL,CASH
750,1,Rates,Option,CapFloor,UPI,QZ9GWMW0F24W,New,NaN,2024-05-01T21:24:17,HRMAMP,...,EURO,Other,European-Call,Physical,USD,USD-SOFR CME Term,1,YEAR,CALL,PHYS
1295,1,Rates,Option,CapFloor,UPI,QZ71R558QM5P,New,NaN,2024-07-10T19:02:08,HRMAMP,...,EURO,Other,European-Call,Physical,USD,USD-SOFR CME Term,2,MNTH,CALL,PHYS
1587,1,Rates,Option,CapFloor,UPI,QZZCCP3KHSBJ,New,NaN,2023-11-13T08:21:44,HRMAMC,...,EURO,Other,European-Call,Cash,USD,USD-SOFR CME Term,10,YEAR,CALL,CASH
1734,1,Rates,Option,CapFloor,UPI,QZGQ8P4M8CPG,New,NaN,2023-11-13T07:11:58,HRMAMC,...,EURO,Other,European-Call,Cash,USD,USD-SOFR CME Term,6,MNTH,CALL,CASH
1761,1,Rates,Option,CapFloor,UPI,QZN535W9QH6B,New,NaN,2023-11-13T07:09:48,HRMAMC,...,EURO,Other,European-Call,Cash,USD,USD-SOFR CME Term,1,DAYS,CALL,CASH


In [10]:
cap_df = format_capfloor_time_and_sales(sdr_df[sdr_df["Unique Product Identifier"].isin(cap_rows["Identifier_UPI"])][capfloor_cols])
cap_df

,Event timestamp,Execution Timestamp,Effective Date,Expiration Date,Notional amount-Leg 1,First exercise date,Option Premium Amount,Strike Price,Unique Product Identifier,UPI FISN,UPI Underlier Name,Forward,Expiry (d),Expiry (m),Expiry (yr),Option Premium Amount / $mm Notational
415,2024-10-31 20:55:20,2024-10-31 20:55:20,2024-11-01,2025-11-01,110000000,2025-11-01,8.464770e+05,0.03500,QZQJWDQ4V0VJ,NA/O Call Epn USD,USD-SOFR CME Term,0.0,365,12.166667,1.000000,7695.245091
416,2024-10-31 20:51:46,2024-10-31 18:54:04,2024-10-09,2025-10-09,14000000,2025-10-09,2.780433e+05,0.02500,QZXNP136XML0,NA/O Call Epn USD,USD-SOFR CME Term,-0.0,365,12.166667,1.000000,19860.235714
594,2024-10-31 21:09:18,2024-10-31 21:09:18,2024-11-01,2026-11-01,80000000,2026-11-01,1.620264e+06,0.03500,QZQJWDQ4V0VJ,NA/O Call Epn USD,USD-SOFR CME Term,0.0,730,24.333333,2.000000,20253.302750
611,2024-10-31 20:48:21,2024-10-31 20:48:21,2024-10-31,2027-05-09,110000000,2027-05-09,1.392412e+05,0.06000,QZQJWDQ4V0VJ,NA/O Call Epn USD,USD-SOFR CME Term,-0.0,920,30.666667,2.520548,1265.829091
612,2024-10-31 20:48:21,2024-10-31 20:48:21,2024-10-31,2027-05-09,37000000,2027-05-09,4.637582e+04,0.06000,QZQJWDQ4V0VJ,NA/O Call Epn USD,USD-SOFR CME Term,-0.0,920,30.666667,2.520548,1253.400541
613,2024-10-31 20:58:08,2024-10-31 20:58:08,2024-11-01,2026-05-01,140000000,2026-05-01,2.232995e+05,0.04600,QZQJWDQ4V0VJ,NA/O Call Epn USD,USD-SOFR CME Term,0.0,546,18.200000,1.495890,1594.996750
624,2024-10-31 19:12:40,2024-10-31 19:12:40,2024-11-01,2025-11-01,110000000,2025-11-01,6.055137e+05,3.75000,QZXNP136XML0,NA/O Call Epn USD,USD-SOFR CME Term,0.0,365,12.166667,1.000000,5504.669545
626,2024-10-31 21:03:28,2024-10-31 21:03:28,2024-11-01,2025-11-03,26000000,2025-11-03,4.319079e+05,0.02500,QZQJWDQ4V0VJ,NA/O Call Epn USD,USD-SOFR CME Term,0.0,367,12.233333,1.005479,16611.841846
724,2024-10-31 19:42:57,2024-10-31 19:42:57,2024-11-01,2025-11-01,150000000,2025-11-01,1.240524e+04,0.05850,QZQJWDQ4V0VJ,NA/O Call Epn USD,USD-SOFR CME Term,0.0,365,12.166667,1.000000,82.701600
732,2024-10-31 20:19:30,2024-10-31 20:19:30,2024-11-01,2027-11-01,470000000,2027-11-01,1.212667e+07,0.03250,QZQJWDQ4V0VJ,NA/O Call Epn USD,USD-SOFR CME Term,0.0,1095,36.500000,3.000000,25801.415106


In [11]:
floor_conds = (
    # USD SOFR OIS COMPOUNDED 1D
    (capfloor_dsb_lookup_df["Derived_CFIOptionStyleandType"] == "European-Put") &
    (capfloor_dsb_lookup_df["Attributes_UnderlyingInstrumentIndex"] == "USD-SOFR-OIS Compound") &
    # (capfloor_dsb_lookup_df["Attributes_UnderlyingInstrumentIndex"] == "USD-SOFR-COMPOUND") &
    # (capfloor_dsb_lookup_df["Attributes_UnderlyingInstrumentIndex"] == "USD-SOFR CME Term") &
    # (capfloor_dsb_lookup_df["Attributes_UnderlyingInstrumentIndexTermValue"] == 1) &
    # (capfloor_dsb_lookup_df["Attributes_UnderlyingInstrumentIndexTermUnit"] == "DAYS") &
    (capfloor_dsb_lookup_df["Attributes_NotionalCurrency"] == "USD") &
    ((capfloor_dsb_lookup_df["Attributes_DeliveryType"] == "CASH") | (capfloor_dsb_lookup_df["Attributes_DeliveryType"] == "PHYS"))
)

floor_rows = capfloor_dsb_lookup_df[floor_conds]
floor_rows

,TemplateVersion,Header_AssetClass,Header_InstrumentType,Header_UseCase,Header_Level,Identifier_UPI,Identifier_Status,Identifier_StatusReason,Identifier_LastUpdateDateTime,Derived_ClassificationType,...,Derived_OptionExerciseStyle,Derived_ValuationMethodorTrigger,Derived_CFIOptionStyleandType,Derived_CFIDeliveryType,Attributes_NotionalCurrency,Attributes_UnderlyingInstrumentIndex,Attributes_UnderlyingInstrumentIndexTermValue,Attributes_UnderlyingInstrumentIndexTermUnit,Attributes_OptionType,Attributes_DeliveryType
87,1,Rates,Option,CapFloor,UPI,QZ9CH1006617,New,NaN,2024-09-12T22:33:12,HRMDMP,...,EURO,Other,European-Put,Physical,USD,USD-SOFR-OIS Compound,10,MNTH,PUTO,PHYS
157,1,Rates,Option,CapFloor,UPI,QZM4SXNGVKLR,New,NaN,2024-09-14T00:18:05,HRMDMP,...,EURO,Other,European-Put,Physical,USD,USD-SOFR-OIS Compound,5,MNTH,PUTO,PHYS
187,1,Rates,Option,CapFloor,UPI,QZ7TMC4XFV63,New,NaN,2024-07-03T17:32:27,HRMDMP,...,EURO,Other,European-Put,Physical,USD,USD-SOFR-OIS Compound,6,MNTH,PUTO,PHYS
340,1,Rates,Option,CapFloor,UPI,QZR0PS9QLJ7L,New,NaN,2024-03-25T15:39:39,HRMDMP,...,EURO,Other,European-Put,Physical,USD,USD-SOFR-OIS Compound,2,YEAR,PUTO,PHYS
600,1,Rates,Option,CapFloor,UPI,QZ122N1X4B4D,New,NaN,2024-01-28T17:54:09,HRMDMC,...,EURO,Other,European-Put,Cash,USD,USD-SOFR-OIS Compound,30,DAYS,PUTO,CASH
603,1,Rates,Option,CapFloor,UPI,QZZQ6TRXB4MW,New,NaN,2024-01-28T17:50:43,HRMDMP,...,EURO,Other,European-Put,Physical,USD,USD-SOFR-OIS Compound,90,DAYS,PUTO,PHYS
605,1,Rates,Option,CapFloor,UPI,QZZ2WGHG9162,New,NaN,2024-01-12T20:38:06,HRMDMP,...,EURO,Other,European-Put,Physical,USD,USD-SOFR-OIS Compound,1,MNTH,PUTO,PHYS
649,1,Rates,Option,CapFloor,UPI,QZ6BS66TZ0Q8,New,NaN,2023-12-21T01:48:09,HRMDMC,...,EURO,Other,European-Put,Cash,USD,USD-SOFR-OIS Compound,2,YEAR,PUTO,CASH
813,1,Rates,Option,CapFloor,UPI,QZWL5Z1F8FBC,New,NaN,2024-01-13T09:54:59,HRMDMC,...,EURO,Other,European-Put,Cash,USD,USD-SOFR-OIS Compound,5,YEAR,PUTO,CASH
932,1,Rates,Option,CapFloor,UPI,QZKT5J741NB0,New,NaN,2023-12-21T01:29:24,HRMDMC,...,EURO,Other,European-Put,Cash,USD,USD-SOFR-OIS Compound,12,WEEK,PUTO,CASH


In [12]:
floor_df = format_capfloor_time_and_sales(sdr_df[sdr_df["Unique Product Identifier"].isin(floor_rows["Identifier_UPI"])][capfloor_cols])
floor_df

,Event timestamp,Execution Timestamp,Effective Date,Expiration Date,Notional amount-Leg 1,First exercise date,Option Premium Amount,Strike Price,Unique Product Identifier,UPI FISN,UPI Underlier Name,Forward,Expiry (d),Expiry (m),Expiry (yr),Option Premium Amount / $mm Notational
327,2024-10-31 13:47:04,2024-10-31 13:47:04,2026-09-10,2027-12-10,25000000,2027-12-10,1.940000e+05,0.0350,QZL9GWT1VKHZ,NA/O P Epn USD,USD-SOFR-OIS Compound,2.0,456,15.200000,1.249315,7760.000000
721,2024-10-31 19:58:18,2024-10-31 19:58:18,2026-11-04,2027-11-04,1000000000,2027-11-04,1.325000e+07,0.0364,QZ7VKKVLNLP0,NA/O P Epn USD,USD-SOFR-OIS Compound,2.0,365,12.166667,1.000000,13250.000000
1085,2024-10-31 20:01:09,2024-10-31 20:01:09,2026-11-04,2027-11-04,500000000,2027-11-04,6.625000e+06,0.0364,QZ7VKKVLNLP0,NA/O P Epn USD,USD-SOFR-OIS Compound,2.0,365,12.166667,1.000000,13250.000000
1738,2024-10-31 13:47:04,2024-10-31 13:47:04,2026-09-10,2027-12-10,25000000,2027-12-10,1.940000e+05,0.0350,QZL9GWT1VKHZ,NA/O P Epn USD,USD-SOFR-OIS Compound,2.0,456,15.200000,1.249315,7760.000000
1832,2024-10-31 19:58:18,2024-10-31 19:58:18,2026-11-04,2027-11-04,1000000000,2027-11-04,1.325000e+07,0.0364,QZ7VKKVLNLP0,NA/O P Epn USD,USD-SOFR-OIS Compound,2.0,365,12.166667,1.000000,13250.000000
1836,2024-10-31 19:58:18,2024-10-31 19:58:18,2026-11-04,2027-11-04,1000000000,2027-11-04,1.325000e+07,0.0364,QZ7VKKVLNLP0,NA/O P Epn USD,USD-SOFR-OIS Compound,2.0,365,12.166667,1.000000,13250.000000
1885,2024-10-31 20:01:09,2024-10-31 20:01:09,2026-11-04,2027-11-04,500000000,2027-11-04,6.625000e+06,0.0364,QZ7VKKVLNLP0,NA/O P Epn USD,USD-SOFR-OIS Compound,2.0,365,12.166667,1.000000,13250.000000
1891,2024-10-31 20:01:09,2024-10-31 20:01:09,2026-11-04,2027-11-04,500000000,2027-11-04,6.625000e+06,0.0364,QZ7VKKVLNLP0,NA/O P Epn USD,USD-SOFR-OIS Compound,2.0,365,12.166667,1.000000,13250.000000
4117,2024-10-31 13:47:04,2024-10-31 13:47:04,2026-09-10,2027-12-10,25000000,2027-12-10,1.940000e+05,0.0350,QZL9GWT1VKHZ,NA/O P Epn USD,USD-SOFR-OIS Compound,2.0,456,15.200000,1.249315,7760.000000
5791,2024-10-31 16:26:19,2024-10-31 16:26:19,2024-11-04,2025-11-04,250000000,2025-11-04,2.000000e+05,0.0312,QZ7VKKVLNLP0,NA/O P Epn USD,USD-SOFR-OIS Compound,0.0,365,12.166667,1.000000,800.000000
